<a href="https://colab.research.google.com/github/saipragna25/deep-learning-vision-classifiers-assignment/blob/main/DL_Vision_Classifiers_Part2_FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image Augmentation

In [ ]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Create an ImageDataGenerator object
datagen = ImageDataGenerator(
    rotation_range=15,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image 
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
)

# fit parameters from data
datagen.fit(x_train)

# configure batch size and retrieve one batch of images
for X_batch, y_batch in datagen.flow(x_train, y_train, batch_size=9):
    # create a grid of 3x3 images
    for i in range(0, 9):
        plt.subplot(330 + 1 + i)
        plt.imshow(X_batch[i], interpolation='none')
    # show the plot
    plt.show()
    break


BiT

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, models

# Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# Reshape the data to include a channel dimension
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

# BiT model URL
bit_url = "https://tfhub.dev/google/bit/m-r50x1/1"
bit = hub.KerasLayer(bit_url, input_shape=(32, 32, 3), trainable=False)  # We keep the BiT model frozen

model_bit = models.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.experimental.preprocessing.Rescaling(1./255),
    layers.experimental.preprocessing.Resizing(32, 32),  # Resize images to match BiT model input shape
    layers.Lambda(lambda x: tf.repeat(x, 3, -1)),  # Repeat grayscale channel to create 3-channel images
    bit,
    layers.Dense(10, activation='softmax')
])

model_bit.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model_bit.fit(x_train, y_train, epochs=5)


4422102/4422102 [==============================] - 1s 0us/step
Epoch 1/5
1875/1875 [==============================] - 589s 306ms/step - loss: 0.7943 - accuracy: 0.7762
Epoch 2/5
1875/1875 [==============================] - 570s 304ms/step - loss: 0.6013 - accuracy: 0.8240
Epoch 3/5
1875/1875 [==============================] - 584s 311ms/step - loss: 0.5678 - accuracy: 0.8356
Epoch 4/5
1501/1875 [=======================>......] - ETA: 1:53 - loss: 0.5309 - accuracy: 0.8427

Mlp-mixer

In [3]:
!pip install mlp-mixer-keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for mlp-mixer-keras: filename=mlp_mixer_keras-0.0.5-py3-none-any.whl size=4668 sha256=2114d559d2b28824de8ccccd3e72100abea3d2bf85c1d261ea17fa7e8067b36f
  Stored in directory: /root/.cache/pip/wheels/aa/c3/43/8a4e96dee3c42c2e8c233e502aa9e8056be939b0e3172aedd9
Successfully built mlp-mixer-keras


In [4]:
from mlp_mixer_keras import MlpMixerModel
import numpy as np
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# Resize the images to 32x32
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])

model = MlpMixerModel(input_shape=x_train.shape[1:],
                      num_classes=len(np.unique(y_train)), 
                      num_blocks=4, 
                      patch_size=8,
                      hidden_dim=32, 
                      tokens_mlp_dim=64,
                      channels_mlp_dim=128,
                      use_softmax=True)

model.compile(loss='sparse_categorical_crossentropy', metrics='acc')
model.fit(x_train, y_train, validation_data=(x_test, y_test))


1875/1875 [==============================] - 79s 36ms/step - loss: 0.7715 - acc: 0.7293 - val_loss: 0.5217 - val_acc: 0.8089
